In [ ]:
# PRE-PROCESSING STEP 1
# WARNING: This cell will take a while to run, and uses a significant amount of memory

from nltk.corpus.reader.bnc import BNCCorpusReader
import os
import json

# load all sentences from corpus
BNC_root_dir = os.path.join('DATA', 'ota_20.500.12024_2554', 'download', 'Texts')
bcr = BNCCorpusReader(BNC_root_dir, fileids=r'[A-K]/\w*/\w*\.xml')
all_sents = bcr.sents()

# filter for sentences ending in periods (to filter out questions, titles, etc.)
sents = list(filter(lambda s: len(s) > 0 and s[-1] == '.', all_sents))

# save list of sentences to JSON file
with open(os.path.join('DATA', 'sents.json'), 'w') as f:
    json.dump(sents, f)

In [ ]:
# PRE-PROCESSING STEP 2
# WARNING: This cell will take a while to run, and uses a significant amount of memory

import os
import json

with open(os.path.join('DATA', 'sents.json'), 'r') as f:
    sents = json.load(f)

all_tokens_list = itertools.chain(sents)
all_tokens_set = set(all_tokens_list)
token_freqs = {token : 0 for token in all_tokens_set}

for token in all_tokens_list:
    token_freqs[token] += 1
    
top_10000 = sorted(token_freqs.items(), key=lambda x: x[1], reverse=True)[:10000]
top_10000_dict = {x[0] : idx for idx, x in enumerate(top_10000)}

numberize = {}
for token in all_tokens_set:
    if token in top_10000:
        numberize[token] = top_10000_dict[token]
    else:
        numberize[token] = 10000

with open(os.path.join('DATA', 'numberize.json'), 'w') as f:
    json.dump(numberize, f)
        
reverse_numberize = {10000 : '<UNK>'}
for token, idx in top_10000_dict.items():
    reverse_numberize[idx] = token
    
with open(os.path.join('DATA', 'reverse_numberize.json'), 'w') as f:
    json.dump(reverse_numberize, f)
